### 1. MongoDB Aggregation Framework
* 기존의 find로는 원하는 데이터로 가공하는데 어려움
* 빅데이터를 다루려면 새로운 데이터 가공 방식이 필요
* 이를 위해 MongoDB 에서는 Aggregation Framework (집계 프레임워크) 를 제공
   - MongoDB Aggregation Framework 는 파이프라인(pipeline) 개념을 모델로 함
   - 파이프라인(pipeline) 이란, 이전 단계의 연산결과를 다음 단계연산에 이용하는 것을 의미
      - Unix의 pipe와 같은 방식으로 데이터를 처리하는 방식
      - document를 여러 단계의 파이프라인으로 처리해서, 데이터를 처리/집계한다고 이해하면 됨
* MongoDB Aggregation Framework 를 사용하면 documents를 grouping, filtering 등 다양한 연산을 적용할 수 있음

> 단순하게, find() 보다 복잡한 검색과 연산을 지원하는 또다른 MongoDB 문법이라고 생각하시면 쉽습니다.

### Shard 와 MongoDB Aggregation Framework(mongoDB가 빅데이터 처리에 좋은이유): 
  - Shard 란 전체 데이터의 서브셋을 가진 서버를 의미함
     - Sharding 은 <여러 장비에 데이터를 분할하는 과정을 의미하고, Partitioning 이라고도 함
  - MongoDB 에서도 Shading 을 통해 여러 장비에 collection 을 쪼개 넣을 수 있고,
  - MongoDB Aggregation Framework 를 통해, 여러 장비에 동일한 요청을 동시에 처리하도록 하고, 
  - 이를 집계하여 계산하면, 빅데이터 상에서 성능을 급격히 높일 수 있음

### MongoDB Aggregation Framework 문법과 Pipeline 동작
- Aggregation Pipeline: Aggregation은 주로 Aggregation Pipeline을 사용하여 데이터를 처리
   - Pipeline은 여러 단계로 구성되며, 각 단계는 데이터를 변형하거나 필터링하는데 사용 
   - **각 단계는 이전 단계의 결과를 입력으로 받아 처리**하고, 최종 결과를 반환


- 매칭, 그룹화, 정렬, 프로젝션: Aggregation Pipeline은 다양한 단계로 구성될 수 있음
   - 주요 단계에는 `$match`, `$group`, `$sort`, `$project` 등이 있음 
      - `$match`는 조건에 맞는 문서를 필터링
      - `$group`은 문서를 그룹화하고 집계 작업을 수행
      - `$sort`는 결과를 정렬
      - `$project`는 필요한 필드를 선택하거나 계산하여 결과에 포함시킴
   - 이외에 다양한 집계 작업을 위한 기능 제공
      - `$sum, $avg, $min, $max, $count` 등으로 필드를 합산하거나 평균을 계산하고, 최소/최대 값을 찾을 수 있음

<center>
<img src="https://davelee-fun.github.io/fixeddata/mongodb_aggregate_pipeline.png" />
이미지 출처 - https://docs.mongodb.com/manual/aggregation/#aggregation-framework
</center>

### 2. 주요 Mongodb Aggregation 문법

### 사전준비
- 제공해드린 데이터를 툴을 사용하여 mongodb 에 import =>DB생성이 아니라 직접 import

### sample_mflix 데이터베이스

- `sample_mflix` 데이터베이스는 여러 영화와 관련 정보가 포함되어 있음 
- 영화 정보, 배우, 감독, 영화 리뷰 등에 대한 데이터가 포함되어 있음

#### movies collection
- 이 컬렉션은 각각의 영화에 대한 정보를 포함하고 있음 
    - `_id`: 각 영화의 고유 식별자
    - `title`: 영화의 제목
    - `plot`: 영화의 줄거리
    - `genres`: 영화의 장르 (예: 액션, 코미디 등)
    - `runtime`: 영화의 상영 시간 (분 단위)
    - `cast`: 영화 출연 배우
    - `directors`: 영화 감독
    - `year`: 영화가 개봉한 년도

#### comments collection
- 이 컬렉션은 사용자들이 남긴 영화에 대한 댓글을 포함하고 있음
    - `_id`: 각 댓글의 고유 식별자
    - `name`: 댓글을 남긴 사용자의 이름
    - `email`: 댓글을 남긴 사용자의 이메일
    - `movie_id`: 댓글이 남겨진 영화의 식별자
    - `text`: 댓글의 내용
    - `date`: 댓글이 작성된 날짜

### aggregate()
- `aggregate()` 메서드는 Mongodb Aggregation 작업을 수행할 수 있게 해주는 메서드
- 이 메서드는 배열 형태로 표현하며, 맨 처음 연산부터, 순차적으로 파이프라인 연산을 수행함

```bash
db.collection.aggregate([ { <stage1> }, { <stage2> }, ... ])
```
------
## aggregate 조건문

### `$match`

- `$match` 단계는 지정된 조건을 만족하는 문서만을 필터링하여 다음 파이프라인 단계로 전달하는 데 사용합니다

```bash
{ $match: { <query> } }
```

- 예를 들어, 1995년에 개봉한 모든 영화를 선택하려고 한다면 아래와 같이 질의할 수 있습니다

```bash
db.movies.aggregate([
  { $match: { year: 1995 } }
]);
```
---------------------------
### `$group`

- `$group` 단계는 Document에 대한 그룹화를 수행하여 복잡한 집계 연산을 가능하게 합니다.
- `$group` 연산자는 그룹의 키를 정의하는 `_id` 필드를 필요로 합니다.
   - `_id`는 표현식이 될 수 있으며, 이는 그룹화의 기준이 됩니다.
- 각 그룹에 대해 집계를 수행할 수 있습니다.
   - 이는 필드와 해당 필드에 적용할 누산식(accumulator)의 쌍으로 제공됩니다.
   - 이 누산식은 `$sum`, `$avg`, `$min`, `$max` 등과 같은 여러 가지 연산자를 포함할 수 있습니다.

```bash
db.collection.aggregate([
  { 
    $group: {
      _id: <expression>, // 그룹화할 필드를 지정
      <field1>: { <accumulator1> : <expression1> }, // 그룹에 적용할 누산식
      ...
    }
  }
]);
```

- 예를 들어, 모든 영화에 대한 댓글 수를 집계하려면 다음과 같이 쿼리를 작성할 수 있습니다.
   - 여기서는 각 영화를 그룹화(`_id: "$movie_id"`)하고, 각 그룹에 대해 댓글 수를 누적(`$sum: 1`)하여 `commentCount` 필드에 저장합니다.

```bash
db.comments.aggregate([
  { 
    $group: {
      _id: "$movie_id",
      commentCount: { $sum: 1 }
    }
  }
]);

{ $match: { "imdb.rating": { $ne: "" or null} } }  $ne 는 "", null값을 제외한 값을 추출한다.
```

- `$sum: 1`은 그룹화된 문서의 개수를 세는 것을 의미합니다. 즉, 그룹 내의 각 문서를 카운트하여 개수를 계산합니다.
- `$sum: "$runtime" ` 과 같이 작성하면, runtime 컬럼의 값의 총합을 의미합니다.
-------------------------

### $group 에 쓰이는 주요 accumulator

**1. `$sum`: 숫자 값을 누적하여 합계를 계산**
   ```bash
   db.movies.aggregate([
     {
       $group: {
         _id: "$year", 
         totalMovies: { $sum: 1 }
       }
     }
   ]);
   ```

   위 예제에서는 각 연도별로 출시된 영화의 총 수(`totalMovies`)를 계산합니다.

**2. `$avg`: 숫자 값의 평균을 계산**

   ```bash
   db.movies.aggregate([
     {
       $group: {
         _id: "$year",
         averageRating: { $avg: "$imdb.rating" }
       }
     }
   ]);
   ```

   위 예제에서는 각 연도별 영화의 IMDB 평점의 평균(`averageRating`)을 계산합니다.

**3. `$min` & `$max`: 최소값과 최대값을 찾음**

   ```bash
   db.movies.aggregate([
     {
       $group: {
         _id: "$year",
         minRating: { $min: "$imdb.rating" },
         maxRating: { $max: "$imdb.rating" }
       }
     }
   ]);
   ```

   위 예제에서는 각 연도별 영화의 IMDB 평점 중 최소(`minRating`)와 최대(`maxRating`) 평점을 찾습니다.

**4. `$push`: 그룹에 있는 모든 값들을 배열로 반환**

```bash
db.movies.aggregate([
 {
   $group: {
     _id: "$year",
     titles: { $push: "$title"}
   }
 }
]);
```

   위 예제에서는 각 연도별로 출시된 영화의 제목들(`titles`)을 배열로 묶습니다.

**5. `$addToSet`: 중복되지 않는 값들만 배열로 반환**

   ```bash
   db.movies.aggregate([
     {
       $group: {
         _id: "$year",
         genres: { $addToSet: "$genres" }
       }
     }
   ]);
   ```

   위 예제에서는 각 연도별 영화 장르(`genres`)를 중복 없이 배열로 묶습니다.


**6. `$first` & `$last`: 그룹에서 가장 첫번째 또는 마지막 문서를 반환**



```bash
db.movies.aggregate([
    {
        $sort: { "year": 1, "title": 1 }
    },
    {
        $group: {
            _id: "$year",
            firstMovie: { $first: "$title" },
            lastMovie: { $last: "$title" }
        }
    }
]);
```

위 예제에서는 각 연도별로 가장 먼저와 마지막으로 등록된 영화의 제목(`firstMovie`, `lastMovie`)을 찾습니다. `$first`와 `$last`는 입력 문서의 순서에 의존하므로, 일반적으로 `$sort` 연산자와 함께 사용됩니다.


**7. `$strLenCP`: 문자열의 길이를 반환**


```bash
db.movies.aggregate([
    {
        $group: {
            _id: "$year",
            avgTitleLength: { $avg: { $strLenCP: { $toString: "$title" } } }
        }
    }
]);
```

위 예제에서는 각 연도별 영화 제목의 평균 길이(`avgTitleLength`)를 계산합니다.

---------------------------

### `$count`

`$count` 스테이지는 파이프라인에서 들어오는 문서 수를 계산합니다. 이 스테이지는 출력 필드의 이름을 파라미터로 받아 출력 문서에 추가합니다.

예시:

2000년 이후에 출시된 영화의 수를 계산하는 쿼리입니다:

```bash
db.movies.aggregate([
    { $match: { year: { $gte: 2000 } } },
    { $count: "movies_since_2000" }
]);
```

위의 쿼리는 2000년 이후에 출시된 영화의 개수를 `movies_since_2000` 필드에 저장하게 됩니다.

-------------------------------

### `$sort`

`$sort` 스테이지는 입력 문서를 지정된 필드를 기준으로 정렬합니다. 정렬 필드와 순서(오름차순 : 1, 내림차순 : -1)를 지정할 수 있습니다.

예시:

영화를 출시 연도와 제목을 기준으로 오름차순으로 정렬하는 쿼리입니다:

```bash
db.movies.aggregate([
    { $sort: { "year": 1, "title": 1 } }
]);
```

위의 쿼리는 먼저 영화를 출시 연도에 따라 정렬한 후, 같은 연도의 영화들은 제목으로 다시 정렬하게 됩니다.

---------------------------------
### `$unwind`

`$unwind` 스테이지는 배열 필드를 "풀어"서 각각의 배열 요소가 개별 문서로 처리될 수 있게 합니다. 배열에 포함된 각 요소에 대해 입력 문서의 복사본을 생성하며, 이렇게 생성된 문서는 배열에서의 요소와 함께 출력됩니다.

예시:

각 영화의 장르별로 문서를 생성하는 쿼리입니다:

```bash
db.movies.aggregate([
    { $unwind: "$genres" }
]);
```

위의 쿼리는 각 영화의 각 장르에 대해 별도의 문서를 생성하게 됩니다. 그 결과, 장르가 'Drama', 'Comedy', 'Romance'인 영화는 세 개의 문서로 처리됩니다. 각 문서는 원래의 문서에서 복사되며, "genres" 필드는 해당 장르로 대체됩니다.

---------------------------------------
### `$limit`

`$limit` 스테이지는 파이프라인이 출력하는 문서의 수를 제한합니다. `$limit`는 숫자를 인자로 받으며, 이 숫자는 출력할 문서의 최대 개수를 지정합니다.

`$limit`는 주로 정렬된 데이터셋에서 상위 혹은 하위 n개의 결과를 가져올 때 사용됩니다. 이 때는 `$sort` 스테이지와 함께 사용됩니다.

예시:

가장 높은 평점을 가진 상위 5개의 영화를 선택하는 쿼리입니다:

```bash
db.movies.aggregate([
    { $sort: { "imdb.rating": -1 } },
    { $limit: 5 }
]);
```

위의 쿼리는 영화를 IMDB 평점에 따라 내림차순으로 정렬한 후, 상위 5개의 영화만 선택하게 됩니다.

`$limit`은 성능 최적화를 위해 자주 사용됩니다. 큰 데이터셋에서 작은 부분집합만 필요할 때 `$limit`를 사용하면 불필요한 데이터 처리를 피할 수 있어 처리 속도를 향상시킬 수 있습니다.

----------------

### aggregation문법 느낌 한줄정리
- 크게 match,group,count,sort,unwind,limit : {}안에서 세부조건 덧붙이기
- 거의 이런느낌 {변수이름:{조건:"$속성명"}},
- match는 걍 조건문
- group에서 select groupby 등이 거의다 이루어짐
- count는 위 파이프라인에서 걸러져나오는 모든 document의 수(튜플의수) 전부 합산(모두더하기)
- sort는 정렬 파이프라인
- limit는 맨 마지막 파이프라인 검색결과 나오는 document의 수를 제한

-----------------------

### 실무문제 part1 

**1. 2000년 이후로 출시된 영화의 수는 얼마인가요?**



**2. 각 연도별로 출시된 영화의 수는 어떻게 되나요?**



**3. 가장 많은 영화가 출시된 연도는 언제인가요?**



**4. 각 연도별 평균 영화 러닝타임은 어떻게 되나요?**



**5. 가장 러닝타임이 긴 영화는 어떤 영화인가요?**



**6. 각 영화 장르별 평균 IMDB 평점은 어떻게 되나요?**



**7. 각 연도별 영화 제목의 평균 길이는 어떻게 되나요?**



**8. 각 연도별로 가장 먼저 출시된 영화의 제목은 무엇인가요?**



**9. 각 연도별로 가장 마지막에 출시된 영화의 제목은 무엇인가요?**



**10. 각 연도별로 고유한 영화 장르는 어떻게 되나요?**


----------------------

### `$project` sql의 view 에서 변환요소 추가 but 원래 컬렉션 자체가 바뀌진 않는다.( ex)concat)

`$project` 스테이지는 출력 문서에 특정 필드를 선택하거나 필드의 형식을 변환하는 데 사용됩니다. 필드 이름에 대한 명시적인 표현식, 새로운 필드 생성, 필드 제외 등 다양한 기능을 제공합니다.

예시:

각 영화의 제목과 출시 연도만 출력하는 쿼리입니다:

```bash
db.movies.aggregate([
    { $project: { _id: 0, title: 1, year: 1 } }
]);
```

위의 쿼리는 출력 문서에서 `_id` 필드를 제외하고, `title`과 `year` 필드만을 선택하여 출력합니다.

새로운 필드를 생성하는 예시로는 다음과 같습니다:

```bash
db.movies.aggregate([
    { 
        $project: { 
            title: 1,
            year: 1,
            releasedIn: { $concat: ["$title", " (", { $toString: "$year" }, ")"] }
        }
    }
]);
```

위의 쿼리는 `title`과 `year` 필드를 선택하고, `releasedIn`이라는 새로운 필드를 생성합니다. `releasedIn` 필드는 `title`과 `year`을 연결하고 괄호로 둘러싸여 출력됩니다.

`$project` 스테이지는 출력에 특정 필드를 포함하거나 제외함으로써 데이터를 가공하는 데 유용합니다. 필요한 필드만 선택하여 데이터의 용량을 줄이거나 필드 형식을 변환하여 데이터를 처리할 수 있습니다


### `$concat` <project나 look up 필드에서 쓰이는 연산자>

`$concat` 연산자는 문자열 필드를 연결하여 새로운 문자열을 생성하는 데 사용됩니다. 여러 문자열 필드를 조합하거나 문자열과 상수를 문자열로 만들어 조합하여 새로운 문자열 값을 만들 수 있습니다.

```bash
{ $concat: [ <expression1>, <expression2>, ... ] }
```
-  \<expression1\>, \<expression2\> 등은 문자열 필드 또는 상수 값입니다.
- 인자로 전달된 필드 또는 상수 값을 순서대로 연결하여 새로운 문자열을 생성합니다.
- `$concat`은 문자열 필드 또는 상수를 조합하여 동적인 문자열 값을 생성하는 데 유용합니다. 필요한 경우 다른 Aggregation 연산자와 함께 사용하여 문자열을 가공하고 다양한 형식으로 표현할 수 있습니다.


예시:

영화 제목과 출시 연도를 조합하여 새로운 필드인 `titleWithYear`를 생성하는 쿼리입니다:

```bash
db.movies.aggregate([
    {
        $project: {
            _id: 0,
            title: 1,
            year: 1,
            titleWithYear: { $concat: ["$title", " (", { $toString: "$year" }, ")"] }
        }
    }
]);
```

위의 쿼리는 `$concat`을 사용하여 `$title` 필드와 " (" 문자열, `$year` 필드를 문자열로 변환한 값, ")" 문자열을 조합하여 `titleWithYear` 필드를 생성합니다. 이를 통해 영화 제목과 출시 연도가 포함된 새로운 문자열 값을 만들 수 있습니다.

------------------------------------

### `$lookup`  sql의 join
- `$lookup` 연산자는 Aggregation Framework에서 다른 컬렉션과의 조인(join) 작업을 수행하여 관련 데이터를 결합하는 데 사용됩니다. 주어진 필드에서 값이 일치하는 문서를 다른 컬렉션에서 찾아서 연결합니다.
- `$lookup` 연산자는 다른 컬렉션과의 조인 작업을 통해 데이터를 결합할 수 있어 복잡한 데이터 질의와 관련된 정보를 가져올 때 유용합니다. 

```bash
{
    $lookup: {
        from: <collection>,
        localField: <field>,
        foreignField: <field>,
        as: <newField>
    }
}
```
- from: 조인할 컬렉션의 이름입니다.
- localField: 현재 컬렉션에서 조인에 사용할 필드입니다.
- foreignField: 조인할 컬렉션에서 조인에 사용할 필드입니다.
- as: 조인된 결과(전체결과)를 저장할 필드의 이름입니다.

예시:

comments 컬렉션의 각 도큐먼트에 대해 movies 컬렉션의 도큐먼트와 일치하는 것을 찾으려면 다음과 같이 쿼리를 작성할 수 있습니다.

```bash
db.comments.aggregate([     comments 컬렉션에 대해
   {
      $lookup:
        {
          from: "movies",   movie컬렉션과 조인
          localField: "movie_id",  movie컬렉션(피 조인 컬렉션) 의 조인조건 컬럼 (sql -> on) 
          foreignField: "_id",   comments 컬렉션 조인조건 컬럼
          as: "movie"  조인된 '전체 결과'에 대한 명칭정의
        }
   }
])

```

위의 쿼리는 comments 컬렉션의 movie_id 필드와 movies 컬렉션의 _id 필드가 일치하는 도큐먼트를 찾습니다. 일치하는 각 movies 도큐먼트는 새로운 movie 필드로 comments 도큐먼트에 추가됩니다.



----------

### 추가적으로... `$skip` , `$redact`

`$skip` 스테이지는 파이프라인에서 일정 개수의 문서를 건너뛰고 그 다음 문서들을 출력합니다.

문법:
```bash
{ $skip: <num> }
```

- `<num>`은 건너뛸 문서의 개수를 나타내는 숫자입니다.

예시:

1. 첫 번째 3개의 영화를 건너뛰고 나머지 영화를 출력하는 쿼리입니다:
```bash
db.movies.aggregate([
    { $skip: 3 }
]);
```

2. 러닝타임이 100분 이상인 영화 리스트에서 상위 5개의 영화를 건너띄고, 나머지 영화를 출력하는 쿼리입니다:
```bash
db.movies.aggregate([
    { $match: { runtime: { $gte: 100 } } },
    { $skip: 5 }
]);
```

-------

`$redact` 스테이지는 문서 내에서 보안이나 필터링 규칙에 따라 문서를 제어하는 데 사용됩니다.

문법:
```bash
{
    $redact: {
        $cond: {
            if: <condition>,
            then: <expression1>,
            else: <expression2>
        }
    }
}
```

- `<condition>`은 문서를 허용할지 거부할지 결정하는 조건

할지 결정하는 조건을 나타내는 표현식입니다.
- `<expression1>`은 조건이 true일 때 적용되는 표현식입니다.
- `<expression2>`는 조건이 false일 때 적용되는 표현식입니다.

예시:

평균 평점이 7 이상인 영화만을 출력하는 쿼리입니다:
```bash
db.movies.aggregate([
    {
        $redact: {
            $cond: {
                if: { $gte: ["$imdb.rating", 7] },
                then: "$$KEEP",
                else: "$$PRUNE"
            }
        }
    }
])
```

- `$$KEEP`: 이 변수를 사용하면 현재 도큐먼트 또는 임베디드 도큐먼트를 보존하고 다음 하위 도큐먼트를 검토합니다.
- `$$PRUNE`: 이 변수를 사용하면 현재 도큐먼트 또는 임베디드 도큐먼트를 제거하고 추가 검토를 중지합니다.

---------------------------------------

### `$facet`

`$facet` 스테이지는 단일 Aggregation 파이프라인 내에서 다중 Aggregation 파이프라인을 정의하여 여러 결과 집합을 생성합니다.

문법:
```bash
{
    $facet: {
        <outputField1>: [ <stage1>, <stage2>, ... ],
        <outputField2>: [ <stage3>, <stage4>, ... ],
        ...
    }
}
```

- `<outputField1>`, `<outputField2>` 등은 각각의 결과 집합에 대한 출력 필드 이름입니다.
- `<stage1>`, `<stage2>` 등은 각각의 결과 집합을 생성하기 위한 Aggregation 스테이지입니다.

예시:

1. 출시 연도별로 영화 수와 최고 평점을 구하는 쿼리입니다:
```bash
db.movies.aggregate([
    {
        $facet: {
            movieCountByYear: [
                { $group: { _id: "$year", count: { $sum: 1 } } }
            ],
            maxRatingByYear: [
                { $group: { _id: "$year", maxRating: { $max: "$imdb.rating" } } }
            ]
        }
    }
]);
```

-------------------------------

### 실습문제 part2
+ 생각해볼만한 내용: project - concat으로 내용을 추가하는 맥락에서, str타입의 컬럼내용을  내용과 내용에 대한 정보 즉 json형식의 데이터로 바꿀수있나
+ 속성명에 달러기호 넣는거 기준: {안에있는 속성은 달러기호 x} :옆에 있는 속성은 달러기호o
**1. 각 영화의 제목과 해당 영화에 달린 댓글들을 출력하세요.**


**2. 평점이 가장 높은 영화의 제목과 평점을 출력하세요.**



**3. 각 장르별로 평균 평점이 가장 높은 장르와 평균 평점을 출력하세요.**


**4. 개봉 연도별로 평균 러닝타임이 가장 짧은 영화의 개봉 연도와 평균 러닝타임을 출력하세요.**


**5. 각 국가별로 가장 많은 영화를 제작한 감독과 그 감독의 영화 수를 출력하세요.**


**6. 각 연도별로 가장 많은 평점을 받은 영화의 제목과 평점을 출력하세요.**


**7. 각 장르별 영화 갯수를 영화 갯수가 가장 많은 순으로 출력하세요.**


**8. 영화 감독별로 평균 평점이 가장 높은 감독과 그 감독의 평균 평점을 출력하세요.**


**9. 장르별로 평균 러닝타임이 가장 긴 장르와 그 장르의 평균 러닝타임을 출력하세요.**


**10. 각 영화의 제목과 해당 영화에 대해 댓글을 남긴 사용자들을 출력하세요.**


4,5